## What is TensorRT?

TensorRT is an optimization tool provided by NVIDIA that applies graph optimization and layer fusion, and finds the fastest implementation of a deep learning model. In other words, TensorRT will optimize our deep learning model so that we expect a faster inference time than the original model (before optimization), such as 5x faster or 2x faster. The bigger model we have, the bigger space for TensorRT to optimize the model. Furthermore, this TensorRT supports all NVIDIA GPU devices, such as 1080Ti, Titan XP for Desktop, and Jetson TX1, TX2 for embedded device.

## Standard workflow for optimizing Tensorflow model to TensorRT

![alt text](pictures/tf-trt_workflow.png)

## Library I use in this video series
Pre-requrement: Install TensorRT by following this tutorial [here](https://medium.com/@ardianumam/installing-tensorrt-in-ubuntu-dekstop-1c7307e1dcf6) for Ubuntu dekstop or [here](https://medium.com/@ardianumam/installing-tensorrt-in-jetson-tx2-8d130c4438f5) for Jetson devices

1. Tensorflow 1.12
2. OpenCV 3.4.5
3. Pillow 5.2.0
4. Numpy 1.15.2
5. Matplotlib 3.0.0

### (a) Read input: Tensorflow model and (b) Convert to frozen model (*.pb)

In [3]:
# import the needed libraries
import tensorflow.compat.v1 as tf
#import tensorflow.python.compiler.tensorrt as trt
from tensorflow.python.platform import gfile
from tensorflow.python.compiler.tensorrt import trt_convert as trt

### (c) Optimize the frozen model to TensorRT graph

In [4]:
# has to be use this setting to make a session for TensorRT optimization
with tf.compat.v1.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.50))) as sess:
    # import the meta graph of the tensorflow model
    #saver = tf.train.import_meta_graph("./model/tensorflow/big/model1.meta")
    saver = tf.train.import_meta_graph("./Tensorflow-TensorRT/model/tensorflow/small/model_small.meta")
    # then, restore the weights to the meta graph
    #saver.restore(sess, "./model/tensorflow/big/model1")
    saver.restore(sess, "./Tensorflow-TensorRT/model/tensorflow/small/model_small")
    
    # specify which tensor output you want to obtain 
    # (correspond to prediction result)
    your_outputs = ["output_tensor/Softmax"]
    
    # convert to frozen model
    frozen_graph = tf.graph_util.convert_variables_to_constants(
        sess, # session
        tf.get_default_graph().as_graph_def(),# graph+weight from the session
        output_node_names=your_outputs)
    #write the TensorRT model to be used later for inference
    with gfile.FastGFile("Tensorflow-TensorRT\model\saved_model.pb",'wb') as f:
        f.write(frozen_graph.SerializeToString())
    print("Frozen model is successfully stored!")

KeyboardInterrupt: 

In [9]:
frozen_graph = "Tensorflow-TensorRT\model\saved_model.pb"
from tensorflow._api.v2.experimental.tensorrt import converter

# convert (optimize) frozen model to TensorRT model
trt_graph = trt.create_inference_graph(
    input_graph_def=frozen_graph,# frozen model
    outputs=["output_tensor/Softmax"],
    max_batch_size=2,# specify your max batch size
    max_workspace_size_bytes=2*(10**9),# specify the max workspace
    precision_mode="FP32") # precision, can be "FP32" (32 floating point precision) or "FP16"

#write the TensorRT model to be used later for inference
with gfile.FastGFile("./TensorRT_model.engine", 'wb') as f:
    f.write(trt_graph.SerializeToString())
print("TensorRT model is successfully stored!")

ImportError: cannot import name 'converter' from 'tensorflow._api.v2.experimental.tensorrt' (c:\Users\bhavy\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow\_api\v2\experimental\tensorrt\__init__.py)

### (optional) Count how many nodes/operations before and after optimization

In [8]:
# check how many ops of the original frozen model
all_nodes = len([1 for n in frozen_graph.node])
print("numb. of all_nodes in frozen graph:", all_nodes)


# check how many ops that is converted to TensorRT engine
trt_engine_nodes = len([1 for n in trt_graph.node if str(n.op) == 'TRTEngineOp'])
print("numb. of trt_engine_nodes in TensorRT graph:", trt_engine_nodes)
all_nodes = len([1 for n in trt_graph.node])
print("numb. of all_nodes in TensorRT graph:", all_nodes)

AttributeError: 'str' object has no attribute 'node'

In [9]:
def read_pb_graph(model):
  with gfile.FastGFile(model,'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
  return graph_def

In [10]:
# variable
TENSORRT_MODEL_PATH = './model/TensorRT-4_model.pb'

graph = tf.Graph()
with graph.as_default():
    with tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.50))) as sess:
        # read TensorRT model
        trt_graph = read_pb_graph(TENSORRT_MODEL_PATH)
        #print()
        # obtain the corresponding input-output tensor
        tf.import_graph_def(trt_graph, name='')
        input = sess.graph.get_tensor_by_name('input_tensor_input:0')
        output = sess.graph.get_tensor_by_name('output_tensor/Softmax:0')
        print(sess.graph.get_all_collection_keys())
        # in this case, it demonstrates to perform inference for 50 times
        total_time = 0; n_time_inference = 50
        out_pred = sess.run(output, feed_dict={input: [[1,2,3,4,5,6,7,8,9,0]]})
        for i in range(n_time_inference):
            t1 = time.time()
            out_pred = sess.run(output, feed_dict={input: [[1,2,3,4,5,6,7,8,9,0]]})
            t2 = time.time()
            delta_time = t2 - t1
            total_time += delta_time
            print("needed time in inference-" + str(i) + ": ", delta_time)
        avg_time_tensorRT = total_time / n_time_inference
        print("average inference time: ", avg_time_tensorRT)

NotFoundError: NewRandomAccessFile failed to Create/Open: ./model/TensorRT-4_model.pb : The system cannot find the path specified.
; No such process

In [ ]:
import tensorflow as tf

In [ ]:
print(tf.__version__)